In [ ]:
from avalanche.benchmarks import nc_benchmark # Note that this is the correct way to import nc_benchmark
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, StreamConfusionMatrix, class_accuracy_metrics, bwt_metrics
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import EWC
import torch
from torch.optim.adamw import AdamW
from torch.utils.data import TensorDataset
import numpy as np
from model import LiteNet
from sklearn.preprocessing import StandardScaler
from utils import log_single_metric_no_viz
from avalanche.training.plugins import LRSchedulerPlugin
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# load Dataset //change to your own dataset
train = np.load('CICIDS2017/train.npy')
test = np.load('CICIDS2017/test.npy')
val = np.load('CICIDS2017/val.npy')
print("Finished loading data")

# Standardize features (fit on train only)
scaler = StandardScaler().fit(train[:, :-1])
train_x = scaler.transform(train[:, :-1])
test_x  = scaler.transform(test[:, :-1])
train_y = train[:, -1].astype(np.int64)
test_y  = test[:, -1].astype(np.int64)
val_x = scaler.transform(val[:, :-1])
val_y = val[:, -1].astype(np.int64)

# Create TensorDataset from standardized arrays
train_dataset = TensorDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y))
test_dataset  = TensorDataset(torch.from_numpy(test_x).float(),  torch.from_numpy(test_y))
val_dataset = TensorDataset(torch.from_numpy(val_x).float(), torch.from_numpy(val_y))

/home/afifhaziq/CLiteNet/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/afifhaziq/CLiteNet/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an 

Using device: cuda
Finished loading data


In [ ]:
# nc_benchmark for class incremental learning 
benchmark = nc_benchmark(
         train_dataset, test_dataset,
         n_experiences=4, # Change to ur own number of experiences (task)
         task_labels=False,
         shuffle=True, seed=1234,
         class_ids_from_zero_in_each_exp=False,   
         fixed_class_order=[0,1,2,3,4,5,6,7]   # Change to ur own classes encoded label   
     )

# REMOVE THIS PART
val_benchmark = nc_benchmark(
        val_dataset, val_dataset,
        n_experiences=4,
        task_labels=False,
        shuffle=True, seed=1234,
        class_ids_from_zero_in_each_exp=False,   
        fixed_class_order=[0,1,2,3,4,5,6,7]      
    )

# Change to ur own model
model = LiteNet(num_classes=benchmark.n_classes)

# REMOVE THIS PART
WandBLogger.log_single_metric = log_single_metric_no_viz

# KEEP THIS PART
logger = [InteractiveLogger()]
logger.append(TextLogger(open('EWC_log.txt', 'w')))

# REMOVE THIS PART
logger.append(WandBLogger(project_name="CL-EWC", 
                          params={"mode": "online"}))

# Performance metrics to evaluate the model
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True),
    loss_metrics(epoch_running=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    timing_metrics(epoch=True, experience=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    loggers=logger
)

# Hyperparameter for training. Tune this accordingly. Ensure you understand each of this component.
criterion = torch.nn.CrossEntropyLoss()

# Tune this one based on ur own model performance during training.
# Can try experimenting with different learning rate and optimizer also.
optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)  


# REMOVE THIS PART
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)
lr_plugin = LRSchedulerPlugin(scheduler=scheduler, metric="val_loss", step_granularity="epoch", reset_scheduler=True, reset_lr=True)

# Change to ur own strategy
cl_strategy = EWC(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    ewc_lambda=5000.0, # Change this params based on ur CL strategy
    mode="online",
    decay_factor=0.9,
    train_mb_size=128, # Change this based on ur own model performance during training.
    train_epochs=20, # Change this based on ur own model performance during training.
    eval_mb_size=256, # Change this based on ur own model performance during training.
    device=device,
    evaluator=eval_plugin,
    eval_every=0,
    plugins=[lr_plugin]
)


wandb: Currently logged in as: afifhaziq (csnet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []

for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    cl_strategy.make_optimizer(reset_optimizer_state=True)
    res = cl_strategy.train(experience, num_workers=8, eval_streams=[val_benchmark.test_stream])
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream, num_workers=16))

# REMOVE THIS PART
import wandb
wandb.finish()

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1]
-- >> Start of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 690/690 [00:50<00:00, 13.54it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 49.2017
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp000/0 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp000/1 = 0.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 185/185 [00:13<00:00, 13.37it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Time_Exp/eval_phase/test_stream/Task000/Exp001 = 13.8066
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp001/0 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp001/1 = 0.0000
	Top1_ClassAcc

wandb: ERROR The nbformat package was not found. It is required to save notebook history.



> Eval on experience 3 (Task 0) from test stream ended.
	Time_Exp/eval_phase/test_stream/Task000/Exp003 = 0.6631
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.9992
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/0 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/1 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/2 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/3 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/4 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/5 = 0.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/6 = 1.0000
	Top1_ClassAcc_Exp/eval_phase/test_stream/Task000/Exp003/7 = 0.9992
-- >> End of eval phase << --
	ConfusionMatrix_Stream/eval_phase/test_stream = 
tensor([[  1881,      0,     85,   4779,    873,      0,    602, 166201],
        [     0,      0,    445,     74,      0,      0,    390,   1150],
        [     5,      0,     32,      2,   1152,     

ExperienceBWT/eval_phase/test_stream/Task000/Exp000,█▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp001,▁█
ExperienceBWT/eval_phase/test_stream/Task000/Exp002,▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp000,▁██
ExperienceForgetting/eval_phase/test_stream/Task000/Exp001,█▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp002,▁
Loss_Stream/eval_phase/test_stream/Task000,▁█▅▄▄
RunningLoss_Epoch/train_phase/train_stream/Task000,▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁█
StreamBWT/eval_phase/test_stream,██▂▁▁
StreamForgetting/eval_phase/test_stream,▁▁▇██
+43,...
